# Aprendizaje de los modelos de aprendizaje automático

Primero importamos las librerías necesarias en el desarrollo de este notbook.

In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.feature_selection import SelectKBest,f_classif,f_regression
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn import set_config
from sklearn.metrics import check_scoring
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.multiclass import unique_labels
import plotly.figure_factory as ff
import joblib

Deshabilitamos los warning futuros de versionamiento de Python.

In [ ]:
warnings.filterwarnings("ignore")

Ponemos el atributo *display* de la configuración al valor *diagram* para poder mostrar los pipelines de preprocesamiento de forma gráfica.

In [ ]:
set_config(display='diagram')

Inicializamos una semilla para que todo el notebook sea reproducible y en cada ejecución los modelos y los resultados obtenidos en los experimentos sean los mismos.

In [ ]:
random_state = 27912

Importamos la base de datos trabajada en el notebook de Comprensión de los datos de la misma forma que lo hicimos en este notebook de Comprensión de los datos, indicando además en este caso un tipo diferente de codificación y el índice de la columna para que todo se cargue de forma correcta.

In [ ]:
bbdd=pd.read_csv("../input/aprendizajedemodelostfg/bbdd.csv",sep=",",encoding='utf-8',index_col=0)
bbdd

## Creación de indicadores

Creamos dos funciones para calcular la ganancia de información entre dos variables para ver la correlación entre cada variable predictora con respecto a la variable objetivo.

In [ ]:
def ent(data):
    prob1 = pd.value_counts(data) / len(data)
    return sum(np.log2(prob1) * prob1 * (-1))

def gain(data,str1,str2):
    e1 = data.groupby(str1).apply(lambda x:ent(x[str2]))
    p1 = pd.value_counts(data[str1]) / len(data[str1])
    e2 = sum(e1 * p1)
    return ent(data[str2]) - e2

Calculamos la ganancia de información de cada variable predictora con respecto a la variable objetivo y guardamos todo en un dataframe.

In [ ]:
resultados = pd.DataFrame()
columnas = bbdd.columns.values
mi=[]

for i in range(len(bbdd.columns.values)):
    mi.append(gain(bbdd,bbdd.columns.values[i],'Resultado'))
    
res = dict(zip(columnas,mi))
del res['Resultado']
data=pd.DataFrame.from_dict(res, orient='index')
data.set_axis(['Ganancia de información'], 
                    axis='columns', inplace=True)
data.sort_values('Ganancia de información',ascending=False)

Primero creamos un dataset que será un subconjunto de la base de datos de partida pero con las variables que vamos a utilizar para el agrupamiento de los datos.

In [ ]:
bbdd_clustering=bbdd[["Puntos totales local","Puntos totales visitante","PG local","PG visitante","PE local","PE visitante","PP local","PP visitante","Goles a favor local","Goles a favor visitante","Goles en contra local", "Goles en contra visitante","Porterías a cero local","Porterías a cero visitante","Total tiros local","Total tiros visitante"]]

Separamos este subconjunto de los datos por ligas para que el agrupamiento sea específico para cada liga.

In [ ]:
bbdd_esp=bbdd_clustering.iloc[0:2280]
bbdd_it=bbdd_clustering.iloc[2280:4560]
bbdd_ale=bbdd_clustering.iloc[4560:6396]
bbdd_fr=bbdd_clustering.iloc[6396:8575]
bbdd_en=bbdd_clustering.iloc[8575:10855]

Realizamos el clustering incremental para cada una de las ligas.

In [ ]:
#Separamos las instancias de la primera temporada para tener una instancia del equipo local y otra del equipo visitante
bbdd_esp_separada=[]
for i in range(380):
        bbdd_esp_separada.append(bbdd_esp.iloc[i,[0,2,4,6,8,10,12,14]])
        bbdd_esp_separada.append(bbdd_esp.iloc[i,[1,3,5,7,9,11,13,15]])
bbdd_esp=bbdd_esp.drop(range(0,380),axis=0)

#Entrenamos el modelo por primera vez con el método partial_fit para poder ir entrenando de poco a poco el modelo
kmeans_esp=MiniBatchKMeans(n_clusters=5,random_state=random_state).partial_fit(bbdd_esp_separada)

columna_predicciones_separarada_esp=[]
#Mientras sigamos teniendo instancias que no hayan sido agrupadas
while(bbdd_esp.empty == False):
    bbdd_esp_separada_paquete=[]
    #Separamos las instancias de las cuatro primeras jornadas o de los partidos que nos queden por agrupar
    if(len(bbdd_esp.index)>=40):
        for i in range(40):
            bbdd_esp_separada_paquete.append(bbdd_esp.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_esp_separada_paquete.append(bbdd_esp.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_esp=bbdd_esp.drop(range(bbdd_esp.index[0],bbdd_esp.index[40]),axis=0)
    else:
        for i in range(len(bbdd_esp.index)):
            bbdd_esp_separada_paquete.append(bbdd_esp.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_esp_separada_paquete.append(bbdd_esp.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_esp=pd.DataFrame()
    #Realizamos las predicciones de las instancias del paquete escogido
    predicciones=kmeans_esp.predict(bbdd_esp_separada_paquete)
    #Añadimos el paquete a las instancias ya agrupadas
    bbdd_esp_separada.extend(bbdd_esp_separada_paquete)
    #Volvemos a entrenar el modelo con las instancias agregadas y los datos anteriores
    kmeans_esp=kmeans_esp.partial_fit(bbdd_esp_separada)
    #Volvemos a obtener las predicciones para el paquete
    predicciones_nuevas=kmeans_esp.predict(bbdd_esp_separada_paquete)
    #Vemos si hay convergencia comparando las predicciones obtenidas antes y después de agregar las instancias del paquete
    while(np.array_equal(predicciones_nuevas,predicciones)==False):
        #Mientras no haya convergencia volvemos a entrenar el modelo y volvemos a sacar las predicciones
        predicciones=predicciones_nuevas
        kmeans_esp=kmeans_esp.partial_fit(bbdd_esp_separada)
        predicciones_nuevas=kmeans_esp.predict(bbdd_esp_separada_paquete)
    #Si hay convergencia, se recigen las predicciones obtenidas para el paquete
    columna_predicciones_separarada_esp.extend(predicciones_nuevas)
#Creamos los arrays para crear los datos de las variables de los clustering de todas las instancias
bbdd_esp_clustering_local=[]
bbdd_esp_clustering_visitante=[]
for i in range(380):
    bbdd_esp_clustering_local.append(np.nan)
    bbdd_esp_clustering_visitante.append(np.nan)
for i in range(380,2280):
    bbdd_esp_clustering_local.append(columna_predicciones_separarada_esp.pop(0))
    bbdd_esp_clustering_visitante.append(columna_predicciones_separarada_esp.pop(0))

In [ ]:
#Separamos las instancias de la primera temporada para tener una instancia del equipo local y otra del equipo visitante
bbdd_it_separada=[]
for i in range(380):
        bbdd_it_separada.append(bbdd_it.iloc[i,[0,2,4,6,8,10,12,14]])
        bbdd_it_separada.append(bbdd_it.iloc[i,[1,3,5,7,9,11,13,15]])
bbdd_it=bbdd_it.drop(range(2280,2660),axis=0)
#Entrenamos el modelo por primera vez con el método partial_fit para poder ir entrenando de poco a poco el modelo
kmeans_it=MiniBatchKMeans(n_clusters=5,random_state=random_state).partial_fit(bbdd_it_separada)

columna_predicciones_separarada_it=[]
#Mientras sigamos teniendo instancias que no hayan sido agrupadas
while(bbdd_it.empty == False):
    bbdd_it_separada_paquete=[]
    if(len(bbdd_it.index)>=40):
        #Separamos las instancias de las cuatro primeras jornadas o de los partidos que nos queden por agrupar
        for i in range(40):
            bbdd_it_separada_paquete.append(bbdd_it.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_it_separada_paquete.append(bbdd_it.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_it=bbdd_it.drop(range(bbdd_it.index[0],bbdd_it.index[40]),axis=0)
    else:
        for i in range(len(bbdd_it.index)):
            bbdd_it_separada_paquete.append(bbdd_it.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_it_separada_paquete.append(bbdd_it.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_it=pd.DataFrame()
    #Realizamos las predicciones de las instancias del paquete escogido
    predicciones=kmeans_it.predict(bbdd_it_separada_paquete)
    #Añadimos el paquete a las instancias ya agrupadas
    bbdd_it_separada.extend(bbdd_it_separada_paquete)
    #Volvemos a entrenar el modelo con las instancias agregadas y los datos anteriores
    kmeans_it=kmeans_it.partial_fit(bbdd_it_separada)
    #Volvemos a obtener las predicciones para el paquete
    predicciones_nuevas=kmeans_it.predict(bbdd_it_separada_paquete)
    #Vemos si hay convergencia comparando las predicciones obtenidas antes y después de agregar las instancias del paquete
    while(np.array_equal(predicciones_nuevas,predicciones)==False):
        #Mientras no haya convergencia volvemos a entrenar el modelo y volvemos a sacar las predicciones
        predicciones=predicciones_nuevas
        kmeans_it=kmeans_it.partial_fit(bbdd_it_separada)
        predicciones_nuevas=kmeans_it.predict(bbdd_it_separada_paquete)
    #Si hay convergencia, se recigen las predicciones obtenidas para el paquete
    columna_predicciones_separarada_it.extend(predicciones_nuevas)

#Creamos los arrays para crear los datos de las variables de los clustering de todas las instancias
bbdd_it_clustering_local=[]
bbdd_it_clustering_visitante=[]
for i in range(380):
    bbdd_it_clustering_local.append(np.nan)
    bbdd_it_clustering_visitante.append(np.nan)
for i in range(380,2280):
    bbdd_it_clustering_local.append(columna_predicciones_separarada_it.pop(0))
    bbdd_it_clustering_visitante.append(columna_predicciones_separarada_it.pop(0))

In [ ]:
#Separamos las instancias de la primera temporada para tener una instancia del equipo local y otra del equipo visitante
bbdd_ale_separada=[]
for i in range(306):
        bbdd_ale_separada.append(bbdd_ale.iloc[i,[0,2,4,6,8,10,12,14]])
        bbdd_ale_separada.append(bbdd_ale.iloc[i,[1,3,5,7,9,11,13,15]])
bbdd_ale=bbdd_ale.drop(range(4560,4866),axis=0)

#Entrenamos el modelo por primera vez con el método partial_fit para poder ir entrenando de poco a poco el modelo
kmeans_ale=MiniBatchKMeans(n_clusters=5,random_state=random_state).partial_fit(bbdd_ale_separada)

columna_predicciones_separarada_ale=[]
#Mientras sigamos teniendo instancias que no hayan sido agrupadas
while(bbdd_ale.empty == False):
    bbdd_ale_separada_paquete=[]
    if(len(bbdd_ale.index)>=36):
        #Separamos las instancias de las cuatro primeras jornadas o de los partidos que nos queden por agrupar
        for i in range(36):
            bbdd_ale_separada_paquete.append(bbdd_ale.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_ale_separada_paquete.append(bbdd_ale.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_ale=bbdd_ale.drop(range(bbdd_ale.index[0],bbdd_ale.index[36]),axis=0)
    else:
        for i in range(len(bbdd_ale.index)):
            bbdd_ale_separada_paquete.append(bbdd_ale.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_ale_separada_paquete.append(bbdd_ale.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_ale=pd.DataFrame()
    
    #Realizamos las predicciones de las instancias del paquete escogido
    predicciones=kmeans_ale.predict(bbdd_ale_separada_paquete)
    #Añadimos el paquete a las instancias ya agrupadas
    bbdd_ale_separada.extend(bbdd_ale_separada_paquete)
    #Volvemos a entrenar el modelo con las instancias agregadas y los datos anteriores
    kmeans_ale=kmeans_ale.partial_fit(bbdd_ale_separada)
    #Volvemos a obtener las predicciones para el paquete
    predicciones_nuevas=kmeans_ale.predict(bbdd_ale_separada_paquete)
    #Vemos si hay convergencia comparando las predicciones obtenidas antes y después de agregar las instancias del paquete
    while(np.array_equal(predicciones_nuevas,predicciones)==False):
        #Mientras no haya convergencia volvemos a entrenar el modelo y volvemos a sacar las predicciones
        predicciones=predicciones_nuevas
        kmeans_ale=kmeans_ale.partial_fit(bbdd_ale_separada)
        predicciones_nuevas=kmeans_ale.predict(bbdd_ale_separada_paquete)
    #Si hay convergencia, se recigen las predicciones obtenidas para el paquete
    columna_predicciones_separarada_ale.extend(predicciones_nuevas)
    
#Creamos los arrays para crear los datos de las variables de los clustering de todas las instancias
bbdd_ale_clustering_local=[]
bbdd_ale_clustering_visitante=[]
for i in range(306):
    bbdd_ale_clustering_local.append(np.nan)
    bbdd_ale_clustering_visitante.append(np.nan)
for i in range(306,1836):
    bbdd_ale_clustering_local.append(columna_predicciones_separarada_ale.pop(0))
    bbdd_ale_clustering_visitante.append(columna_predicciones_separarada_ale.pop(0))

In [ ]:
#Separamos las instancias de la primera temporada para tener una instancia del equipo local y otra del equipo visitante
bbdd_fr_separada=[]
for i in range(380):
        bbdd_fr_separada.append(bbdd_fr.iloc[i,[0,2,4,6,8,10,12,14]])
        bbdd_fr_separada.append(bbdd_fr.iloc[i,[1,3,5,7,9,11,13,15]])
bbdd_fr=bbdd_fr.drop(range(6396,6776),axis=0)

#Entrenamos el modelo por primera vez con el método partial_fit para poder ir entrenando de poco a poco el modelo
kmeans_fr=MiniBatchKMeans(n_clusters=5,random_state=random_state).partial_fit(bbdd_fr_separada)

columna_predicciones_separarada_fr=[]
#Mientras sigamos teniendo instancias que no hayan sido agrupadas
while(bbdd_fr.empty == False):
    bbdd_fr_separada_paquete=[]
    if(len(bbdd_fr.index)>=40):
        for i in range(40):
            #Separamos las instancias de las cuatro primeras jornadas o de los partidos que nos queden por agrupar
            bbdd_fr_separada_paquete.append(bbdd_fr.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_fr_separada_paquete.append(bbdd_fr.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_fr=bbdd_fr.drop(range(bbdd_fr.index[0],bbdd_fr.index[40]),axis=0)
    else:
        for i in range(len(bbdd_fr.index)):
            bbdd_fr_separada_paquete.append(bbdd_fr.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_fr_separada_paquete.append(bbdd_fr.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_fr=pd.DataFrame()
    
    #Realizamos las predicciones de las instancias del paquete escogido
    predicciones=kmeans_fr.predict(bbdd_fr_separada_paquete)
    #Añadimos el paquete a las instancias ya agrupadas
    bbdd_fr_separada.extend(bbdd_fr_separada_paquete)
    #Volvemos a entrenar el modelo con las instancias agregadas y los datos anteriores
    kmeans_fr=kmeans_fr.partial_fit(bbdd_fr_separada)
    #Volvemos a obtener las predicciones para el paquete
    predicciones_nuevas=kmeans_fr.predict(bbdd_fr_separada_paquete)
    #Vemos si hay convergencia comparando las predicciones obtenidas antes y después de agregar las instancias del paquete
    while(np.array_equal(predicciones_nuevas,predicciones)==False):
        #Mientras no haya convergencia volvemos a entrenar el modelo y volvemos a sacar las predicciones
        predicciones=predicciones_nuevas
        kmeans_fr=kmeans_fr.partial_fit(bbdd_fr_separada)
        predicciones_nuevas=kmeans_fr.predict(bbdd_fr_separada_paquete)
    #Si hay convergencia, se recigen las predicciones obtenidas para el paquete
    columna_predicciones_separarada_fr.extend(predicciones_nuevas)

#Creamos los arrays para crear los datos de las variables de los clustering de todas las instancias
bbdd_fr_clustering_local=[]
bbdd_fr_clustering_visitante=[]
for i in range(380):
    bbdd_fr_clustering_local.append(np.nan)
    bbdd_fr_clustering_visitante.append(np.nan)
for i in range(380,2179):
    bbdd_fr_clustering_local.append(columna_predicciones_separarada_fr.pop(0))
    bbdd_fr_clustering_visitante.append(columna_predicciones_separarada_fr.pop(0))

In [ ]:
#Separamos las instancias de la primera temporada para tener una instancia del equipo local y otra del equipo visitante
bbdd_en_separada=[]
for i in range(380):
        bbdd_en_separada.append(bbdd_en.iloc[i,[0,2,4,6,8,10,12,14]])
        bbdd_en_separada.append(bbdd_en.iloc[i,[1,3,5,7,9,11,13,15]])
bbdd_en=bbdd_en.drop(range(8575,8955),axis=0)

#Entrenamos el modelo por primera vez con el método partial_fit para poder ir entrenando de poco a poco el modelo
kmeans_en=MiniBatchKMeans(n_clusters=5,random_state=random_state).partial_fit(bbdd_en_separada)

columna_predicciones_separarada_en=[]
#Mientras sigamos teniendo instancias que no hayan sido agrupadas
while(bbdd_en.empty == False):
    bbdd_en_separada_paquete=[]
    if(len(bbdd_en.index)>=40):
        #Separamos las instancias de las cuatro primeras jornadas o de los partidos que nos queden por agrupar
        for i in range(40):
            bbdd_en_separada_paquete.append(bbdd_en.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_en_separada_paquete.append(bbdd_en.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_en=bbdd_en.drop(range(bbdd_en.index[0],bbdd_en.index[40]),axis=0)
    else:
        for i in range(len(bbdd_en.index)):
            bbdd_en_separada_paquete.append(bbdd_en.iloc[i,[0,2,4,6,8,10,12,14]])
            bbdd_en_separada_paquete.append(bbdd_en.iloc[i,[1,3,5,7,9,11,13,15]])
        bbdd_en=pd.DataFrame()
    #Realizamos las predicciones de las instancias del paquete escogido
    predicciones=kmeans_en.predict(bbdd_en_separada_paquete)
    #Añadimos el paquete a las instancias ya agrupadas
    bbdd_en_separada.extend(bbdd_en_separada_paquete)
    #Volvemos a entrenar el modelo con las instancias agregadas y los datos anteriores
    kmeans_en=kmeans_en.partial_fit(bbdd_en_separada)
    #Volvemos a obtener las predicciones para el paquete
    predicciones_nuevas=kmeans_en.predict(bbdd_en_separada_paquete)
    #Vemos si hay convergencia comparando las predicciones obtenidas antes y después de agregar las instancias del paquete
    while(np.array_equal(predicciones_nuevas,predicciones)==False):
        #Mientras no haya convergencia volvemos a entrenar el modelo y volvemos a sacar las predicciones
        predicciones=predicciones_nuevas
        kmeans_en=kmeans_en.partial_fit(bbdd_en_separada)
        predicciones_nuevas=kmeans_en.predict(bbdd_en_separada_paquete)
    #Si hay convergencia, se recigen las predicciones obtenidas para el paquete
    columna_predicciones_separarada_en.extend(predicciones_nuevas)

#Creamos los arrays para crear los datos de las variables de los clustering de todas las instancias
bbdd_en_clustering_local=[]
bbdd_en_clustering_visitante=[]
for i in range(380):
    bbdd_en_clustering_local.append(np.nan)
    bbdd_en_clustering_visitante.append(np.nan)
for i in range(380,2280):
    bbdd_en_clustering_local.append(columna_predicciones_separarada_en.pop(0))
    bbdd_en_clustering_visitante.append(columna_predicciones_separarada_en.pop(0))

Juntamos los valores de las variables del clustering de cada una de las ligas y añadimos las variables con los valores al conjunto de datos.

In [ ]:
bbdd_clustering_local=np.concatenate((bbdd_esp_clustering_local,bbdd_it_clustering_local,bbdd_ale_clustering_local,bbdd_fr_clustering_local,bbdd_en_clustering_local),axis=0)
bbdd_clustering_visitante=np.concatenate((bbdd_esp_clustering_visitante,bbdd_it_clustering_visitante,bbdd_ale_clustering_visitante,bbdd_fr_clustering_visitante,bbdd_en_clustering_visitante),axis=0)

bbdd['Clustering local']=bbdd_clustering_local
bbdd['Clustering visitante']=bbdd_clustering_visitante

Calculamos de nuevo la ganancia de información de cada variable predictora con respecto a la variable objetivo y guardamos todo en un dataframe para ver como de correlacionadas están las dos variables respecto a la variable objetivo.

In [ ]:
resultados = pd.DataFrame()
columnas = bbdd.columns.values
mi=[]

for i in range(len(bbdd.columns.values)):
    mi.append(gain(bbdd,bbdd.columns.values[i],'Resultado'))

res = dict(zip(columnas,mi))
del res['Resultado']
data=pd.DataFrame.from_dict(res, orient='index')
data.set_axis(['Ganancia de información'], axis='columns', inplace=True)
data.sort_values('Ganancia de información',ascending=False)

## Selección de los datos de prueba

Creamos el conjunto de datos de aprendizaje como un subconjunto del conjunto de datos inicial excluyendo variables innecesarias.

In [ ]:
bbdd_aprendizaje=bbdd.drop(columns=['Fecha', 'Marcador', 'Goles equipo local', 'Goles equipo visitante', 'Diferencia de goles', 'Puntos equipo local', 'Puntos equipo visitante'])

Separamos el conjunto de datos por ligas, teniendo un conjunto de datos por cada una de las ligas.

In [ ]:
bbdd_esp=bbdd_aprendizaje.iloc[0:2280]
bbdd_it=bbdd_aprendizaje.iloc[2280:4560]
bbdd_ale=bbdd_aprendizaje.iloc[4560:6396]
bbdd_fr=bbdd_aprendizaje.iloc[6396:8575]
bbdd_en=bbdd_aprendizaje.iloc[8575:10855]

Separamos el conjunto de datos de cada liga en los suconjuntos de train, validation y test.

In [ ]:
bbdd_esp_train=bbdd_esp.iloc[0:1520]
bbdd_esp_validation=bbdd_esp.iloc[1520:1900]
bbdd_esp_test=bbdd_esp.iloc[1900:2280]

In [ ]:
bbdd_it_train=bbdd_it.iloc[0:1520]
bbdd_it_validation=bbdd_it.iloc[1520:1900]
bbdd_it_test=bbdd_it.iloc[1900:2280]

In [ ]:
bbdd_ale_train=bbdd_ale.iloc[0:1224]
bbdd_ale_validation=bbdd_ale.iloc[1224:1530]
bbdd_ale_test=bbdd_ale.iloc[1530:1836]

In [ ]:
bbdd_fr_train=bbdd_fr.iloc[0:1520]
bbdd_fr_validation=bbdd_fr.iloc[1520:1799]
bbdd_fr_test=bbdd_fr.iloc[1799:2179]

In [ ]:
bbdd_en_train=bbdd_en.iloc[0:1520]
bbdd_en_validation=bbdd_en.iloc[1520:1900]
bbdd_en_test=bbdd_en.iloc[1900:2280]

## Limpieza de los datos, transformación de los datos y selección de las técnicas de modelado

Seleccionamos las variables numéricas y categóricas para hacer en cada variable las transformaciones necesarias.

In [ ]:
numeric_features = list(bbdd_aprendizaje.select_dtypes(include=['float64', 'int64']).columns)
numeric_features.remove("Año")
numeric_features.remove("Resultado")
categorical_features = ["País", "Equipo local", "Equipo visitante", "Últimos partidos local", "Últimos partidos visitante"]

Creamos el primer transformador que será un imputador de valores perdidos.

In [ ]:
trf1 = ColumnTransformer(transformers =[ 
    ('cat', SimpleImputer(strategy ='most_frequent'), categorical_features), 
    ('num', KNNImputer(n_neighbors=3), numeric_features), 
], remainder ='passthrough')

Creamos el transformador de la codificación de las variables categóricas.

In [ ]:
trf2 = ColumnTransformer(transformers =[ 
    ('enc', OneHotEncoder(handle_unknown="ignore", sparse = False), list(range(0,5))), 
], remainder ='passthrough')

Creamos el transformador de la codificación de las variables categóricas cuando primero se aplica el transformador de la normalización.

In [ ]:
trf2_norm = ColumnTransformer(transformers =[ 
    ('enc', OneHotEncoder(handle_unknown="ignore", sparse = False), list(range(34,39))), 
], remainder ='passthrough')

Se crea el transformador de la normalización.

In [ ]:
trf_norm = ColumnTransformer(transformers =[ 
    ('norm', MinMaxScaler(), list(range(5,len(bbdd_esp_train.drop(columns=["Resultado"]).iloc[0])-2))), 
], remainder ='passthrough') 

Para cada uno de los modelos, se inicializa el modelo y se incluye en el pipeline personalizado con las transformaciones necesarias en cada caso.

In [ ]:
naive_bayes_model = GaussianNB()

In [ ]:
pipeline_nvm = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2), 
    ('kbest', SelectKBest()),
    ('NVM', naive_bayes_model),
])
pipeline_nvm

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

In [ ]:
pipeline_adaboost = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2), 
    ('kbest', SelectKBest()),
    ('adaboost', adaboost_model),
])
pipeline_adaboost

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

In [ ]:
pipeline_bagging = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2), 
    ('kbest', SelectKBest()),
    ('bagging', bagging_model),
])
pipeline_bagging

In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

In [ ]:
pipeline_rf = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2),
    ('kbest', SelectKBest()),
    ('rf', random_forest_model),
])
pipeline_rf

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

In [ ]:
pipeline_gtb = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2), 
    ('kbest', SelectKBest()),
    ('gtb', gradient_boosting_model),
])
pipeline_gtb

In [ ]:
mlp_model=MLPClassifier(random_state=random_state)

In [ ]:
pipeline_mlp = Pipeline(steps =[ 
    ('tf1', trf1),
    ('trf_norm', trf_norm),
    ('trf2_norm', trf2_norm), 
    ('kbest', SelectKBest()),
    ('mlp', mlp_model),
])
pipeline_mlp

In [ ]:
logistic_regression_model=LogisticRegression(random_state=random_state,multi_class='multinomial')

In [ ]:
pipeline_lrm = Pipeline(steps =[ 
    ('tf1', trf1),
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('lrm', logistic_regression_model),
])
pipeline_lrm

In [ ]:
cv = KFold(n_splits=5)

## Obtención de los modelos

Creamos la función de optimizar parámetros para utilizarla con todos los modelos.

In [ ]:
def optimize_params(estimator, X, y, cv, scoring=None, refit=True, **param_grid):
    #Se realiza la búsqueda en malla con el estimador correspondiente, los datos correspondientes y la validación cruzada
    grid_search_cv = GridSearchCV(estimator,
                                  param_grid,
                                  scoring=scoring,
                                  refit=refit,
                                  cv=cv,
                                  return_train_score=True).fit(X, y)
    
    #Se añaden los resultados a un dataframe
    cv_results = pd.DataFrame(grid_search_cv.cv_results_)
    #Se ponen las etiquetas y se filtran los registros con la variable "rank_test"
    labels = cv_results.filter(regex="split")
    by = cv_results.filter(regex="rank_test").columns[0]
    cv_results = cv_results.drop(labels, axis=1).sort_values(by)

    display(cv_results)
    
    return grid_search_cv

Para cada modelo y para cada liga, se llama a la función creada para optimizar los hiperparámetros y obtener la mejor selección de hiperparámetros.

In [ ]:
naive_bayes_classifier_esp = optimize_params(pipeline_nvm, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, kbest__k=range(10,20))

In [ ]:
naive_bayes_classifier_it = optimize_params(pipeline_nvm, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, kbest__k=range(10,20))

In [ ]:
naive_bayes_classifier_ale = optimize_params(pipeline_nvm, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, kbest__k=range(10,20))

In [ ]:
naive_bayes_classifier_fr = optimize_params(pipeline_nvm, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, kbest__k=range(10,20))

In [ ]:
naive_bayes_classifier_en = optimize_params(pipeline_nvm, bbdd_en_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, kbest__k=range(10,20))

In [ ]:
criterion = ["gini", "entropy"]

In [ ]:
base_estimator = [DecisionTreeClassifier(random_state=random_state)]

In [ ]:
n_estimators = [20, 50, 100]

In [ ]:
learning_rate = [0.95, 1.0]

In [ ]:
max_depth = [2, 3]

In [ ]:
adaboost_classifier_esp = optimize_params(pipeline_adaboost, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
adaboost_classifier_it = optimize_params(pipeline_adaboost, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
adaboost_classifier_ale = optimize_params(pipeline_adaboost, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
adaboost_classifier_fr = optimize_params(pipeline_adaboost, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
adaboost_classifier_en = optimize_params(pipeline_adaboost, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
bagging_classifier_esp = optimize_params(pipeline_bagging, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
bagging_classifier_it = optimize_params(pipeline_bagging, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
bagging_classifier_ale = optimize_params(pipeline_bagging, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
bagging_classifier_fr = optimize_params(pipeline_bagging, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
bagging_classifier_en = optimize_params(pipeline_bagging, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

In [ ]:
max_features = ["sqrt", "log2"]

In [ ]:
random_forest_classifier_esp = optimize_params(pipeline_rf, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

In [ ]:
random_forest_classifier_it = optimize_params(pipeline_rf, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

In [ ]:
random_forest_classifier_ale = optimize_params(pipeline_rf, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

In [ ]:
random_forest_classifier_fr = optimize_params(pipeline_rf, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

In [ ]:
random_forest_classifier_en = optimize_params(pipeline_rf, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

In [ ]:
learning_rate = [0.01, 0.05, 0.1]

In [ ]:
criterion = ["friedman_mse", "squared_error"]

In [ ]:
gradient_boosting_classifier_esp = optimize_params(pipeline_gtb, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

In [ ]:
gradient_boosting_classifier_it = optimize_params(pipeline_gtb, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

In [ ]:
gradient_boosting_classifier_ale = optimize_params(pipeline_gtb, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

In [ ]:
gradient_boosting_classifier_fr = optimize_params(pipeline_gtb, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

In [ ]:
gradient_boosting_classifier_en = optimize_params(pipeline_gtb, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

In [ ]:
activation=['identity','logistic','tanh','relu']

In [ ]:
mlp_classifier_esp = optimize_params(pipeline_mlp, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv,mlp__activation=activation, kbest__k=range(10,20))

In [ ]:
mlp_classifier_it = optimize_params(pipeline_mlp, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv,mlp__activation=activation,kbest__k=range(10,20))

In [ ]:
mlp_classifier_ale = optimize_params(pipeline_mlp, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv,mlp__activation=activation, kbest__k=range(10,20))

In [ ]:
mlp_classifier_fr = optimize_params(pipeline_mlp, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv,mlp__activation=activation, kbest__k=range(10,20))

In [ ]:
mlp_classifier_en = optimize_params(pipeline_mlp, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv,mlp__activation=activation, kbest__k=range(10,20))

In [ ]:
C=[0.8,0.9,1]

In [ ]:
lrm_classifier_esp = optimize_params(pipeline_lrm, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, lrm__C=C, kbest__k=range(10,20))

In [ ]:
lrm_classifier_it = optimize_params(pipeline_lrm, bbdd_it_train.drop(columns=["Resultado"]), bbdd_it_train["Resultado"], cv, lrm__C=C, kbest__k=range(10,20))

In [ ]:
lrm_classifier_ale = optimize_params(pipeline_lrm, bbdd_ale_train.drop(columns=["Resultado"]), bbdd_ale_train["Resultado"], cv, lrm__C=C, kbest__k=range(10,20))

In [ ]:
lrm_classifier_fr = optimize_params(pipeline_lrm, bbdd_fr_train.drop(columns=["Resultado"]), bbdd_fr_train["Resultado"], cv, lrm__C=C, kbest__k=range(10,20))

In [ ]:
lrm_classifier_en = optimize_params(pipeline_lrm, bbdd_en_train.drop(columns=["Resultado"]), bbdd_en_train["Resultado"], cv, lrm__C=C, kbest__k=range(10,20))

## Evaluación de los modelos

Se crean las funciones respectivamente de validación y test para los modelos.

In [ ]:
def evaluate_estimators(estimators, X, y):
    #Se crea un dataframe con una columna por cada métrica a estudiar
    results = pd.DataFrame(columns=["Accuracy","F1","Kappa"])
    #Para cada estimador o modelo estudiado
    for estimator in estimators:
        #Obtenemos su nombre
        name = estimator.estimator[-1].__class__.__name__
        #Predecimos el conjunto de datos de validación
        y_pred = estimator.predict(X)
        
        #Obtenemos las métricas a estudiar entre las predicciones y los valores reales y las añadimos al dataframe
        accuracy=accuracy_score(y,y_pred)

        results.loc[name, "Accuracy"] = accuracy
        
        f1=f1_score(y,y_pred,average="weighted")

        results.loc[name, "F1"] = f1
        
        kappa=cohen_kappa_score(y,y_pred)
        
        results.loc[name, "Kappa"] = kappa
        
    #Devolvemos el dataframe
    return results

def test_evaluate(estimator, X_test, y_test):
    #Obtenemos las predicciones del conjunto de datos de test
    predictions = estimator.predict(X_test)
    
    #Obtenemos el informe de clasificación con las predicciones y los valores reales
    report = classification_report(y_test, predictions, output_dict=True)
    
    #Incluimos el informe en un dataframe
    report = pd.DataFrame(report).T
    
    #Creamos la matriz de confusión con las predicciones y los valores reales
    matrix = confusion_matrix(y_test, predictions)
    
    #Guardamos los valores únicos de las predicciones y los valores reales
    out = unique_labels(y_test, predictions)

    x = y = list(out)
    
    #Creamos la matriz como un mapa de calor anotado con la matriz de confusión obtenida anteriormente
    matrix = ff.create_annotated_heatmap(matrix, x=x, y=y)
    
    for i in range(len(matrix.layout.annotations)):
        matrix.layout.annotations[i].font.size = 18

    xaxis = {"title": "Estimado por el modelo "}

    yaxis = {"title": "Valor real"}
    
    #Asignamos los titulos del mapa de calor anotado
    matrix.update_layout(xaxis=xaxis, yaxis=yaxis)
    
    #Devolvemos el informe y la matriz como mapa de calor anotado
    return report, matrix

Para cada liga se establecen los modelos utilizados para pasarlos a la función de validación.

In [ ]:
estimators_esp = [naive_bayes_classifier_esp, adaboost_classifier_esp, bagging_classifier_esp, random_forest_classifier_esp, gradient_boosting_classifier_esp, mlp_classifier_esp, lrm_classifier_esp]

In [ ]:
estimators_it = [naive_bayes_classifier_it, adaboost_classifier_it, bagging_classifier_it, random_forest_classifier_it, gradient_boosting_classifier_it, mlp_classifier_it, lrm_classifier_it]

In [ ]:
estimators_ale = [naive_bayes_classifier_ale, adaboost_classifier_ale, bagging_classifier_ale, random_forest_classifier_ale, gradient_boosting_classifier_ale, mlp_classifier_ale, lrm_classifier_ale]

In [ ]:
estimators_fr = [naive_bayes_classifier_fr, adaboost_classifier_fr, bagging_classifier_fr, random_forest_classifier_fr, gradient_boosting_classifier_fr, mlp_classifier_fr, lrm_classifier_fr]

In [ ]:
estimators_en = [naive_bayes_classifier_en, adaboost_classifier_en, bagging_classifier_en, random_forest_classifier_en, gradient_boosting_classifier_en, mlp_classifier_en, lrm_classifier_en]

Por cada liga, llamamos primero a la función de validación, luego a la función de test y mostramos todos los resultados.

In [ ]:
evaluate_estimators(estimators_esp, bbdd_esp_validation.drop(columns=["Resultado"]), bbdd_esp_validation["Resultado"])

In [ ]:
esp_test_report, esp_test_matrix = test_evaluate(lrm_classifier_esp, bbdd_esp_test.drop(columns=["Resultado"]), bbdd_esp_test["Resultado"])

In [ ]:
esp_test_report

In [ ]:
esp_test_matrix

In [ ]:
evaluate_estimators(estimators_it, bbdd_it_validation.drop(columns=["Resultado"]), bbdd_it_validation["Resultado"])

In [ ]:
it_test_report, it_test_matrix = test_evaluate(lrm_classifier_it, bbdd_it_test.drop(columns=["Resultado"]), bbdd_it_test["Resultado"])

In [ ]:
it_test_report

In [ ]:
it_test_matrix

In [ ]:
evaluate_estimators(estimators_ale, bbdd_ale_validation.drop(columns=["Resultado"]), bbdd_ale_validation["Resultado"])

In [ ]:
ale_test_report, ale_test_matrix = test_evaluate(mlp_classifier_ale, bbdd_ale_test.drop(columns=["Resultado"]), bbdd_ale_test["Resultado"])

In [ ]:
ale_test_report

In [ ]:
ale_test_matrix

In [ ]:
evaluate_estimators(estimators_fr, bbdd_fr_validation.drop(columns=["Resultado"]), bbdd_fr_validation["Resultado"])

In [ ]:
fr_test_report, fr_test_matrix = test_evaluate(mlp_classifier_fr, bbdd_fr_test.drop(columns=["Resultado"]), bbdd_fr_test["Resultado"])

In [ ]:
fr_test_report

In [ ]:
fr_test_matrix

In [ ]:
evaluate_estimators(estimators_en, bbdd_en_validation.drop(columns=["Resultado"]), bbdd_en_validation["Resultado"])

In [ ]:
en_test_report, en_test_matrix = test_evaluate(mlp_classifier_en, bbdd_en_test.drop(columns=["Resultado"]), bbdd_en_test["Resultado"])

In [ ]:
en_test_report

In [ ]:
en_test_matrix

Guardamos los conjuntos de datos de cada liga y los mejores modelos de cada liga para exportarlos y usarlos en la página web.

In [ ]:
bbdd_esp.to_csv('bbdd_esp.csv',encoding='utf-8')

In [ ]:
joblib.dump(lrm_classifier_esp, 'modelo_entrenado_españa.pkl')

In [ ]:
bbdd_it.to_csv('bbdd_it.csv',encoding='utf-8')

In [ ]:
joblib.dump(lrm_classifier_it, 'modelo_entrenado_italia.pkl')

In [ ]:
bbdd_ale.to_csv('bbdd_ale.csv',encoding='utf-8')

In [ ]:
joblib.dump(mlp_classifier_ale, 'modelo_entrenado_alemania.pkl')

In [ ]:
bbdd_fr.to_csv('bbdd_fr.csv',encoding='utf-8')

In [ ]:
joblib.dump(mlp_classifier_fr, 'modelo_entrenado_francia.pkl')

In [ ]:
bbdd_en.to_csv('bbdd_en.csv',encoding='utf-8')

In [ ]:
joblib.dump(mlp_classifier_en, 'modelo_entrenado_inglaterra.pkl')

Guardamos el conjunto de datos final.

In [ ]:
bbdd.to_csv('bbdd_final.csv',encoding='utf-8')